In [18]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
from json import load


In [19]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [20]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [21]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [22]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [23]:
import tensorflow as tf

model_class = tf.keras.models.Sequential()
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='relu',
                                      input_shape=(X_train[0].shape[0],), name='input'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*3), activation='relu', name='hidden_layer_3'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*2), activation='sigmoid', name='hidden_layer_4', use_bias=True))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*1.5), activation='sigmoid', name='hidden_layer_5', use_bias=True))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*0.5), activation='sigmoid', name='hidden_layer_6', use_bias=True))
model_class.add(tf.keras.layers.Dense(1, name='output'))
model_class.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 hidden_layer_3 (Dense)      (None, 288)               55584     
                                                                 
 hidden_layer_4 (Dense)      (None, 192)               55488     
                                                                 
 hidden_layer_5 (Dense)      (None, 144)               27792     
                                                                 
 hidden_layer_6 (Dense)      (None, 48)                6960      
                                                                 
 output (Dense)              (None, 1)                

In [24]:
model_class.compile(
    loss='mse',  # Функция потерь
    optimizer='Adam',  # Оптимизатор
    metrics=[  # Метрики
        'mse',  # Если у объекта назначено имя, то можно вызвать объект с его помощью
        # tf.keras.metrics.TrueNegatives
        # tf.keras.metrics.Precision()
    ]
)


In [25]:
model_class.fit(
    X_train,  # Набор входных данных
    y_train,  # Набор правильных ответов
    validation_split=0.2,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=40,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=2
)


Epoch 1/40
319/319 [==============================] - 2s 3ms/step - loss: 0.1288 - mse: 0.1288 - val_loss: 0.0642 - val_mse: 0.0642
Epoch 2/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0645 - mse: 0.0645 - val_loss: 0.0752 - val_mse: 0.0752
Epoch 3/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0428 - mse: 0.0428 - val_loss: 0.0530 - val_mse: 0.0530
Epoch 4/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0312 - mse: 0.0312 - val_loss: 0.0512 - val_mse: 0.0512
Epoch 5/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.0451 - val_mse: 0.0451
Epoch 6/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.0431 - val_mse: 0.0431
Epoch 7/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.0511 - val_mse: 0.0511
Epoch 8/40
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [26]:
predict = model_class.predict(X_test)


9/9 [==============================] - 0s 1ms/step


In [27]:
mean_squared_error(predict, y_test)


0.04263766919176076

In [28]:
predictA = model_class.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9812206572769953
Confusion matrix
[[346  12]
 [  8 699]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       358
           1       0.98      0.99      0.99       707

    accuracy                           0.98      1065
   macro avg       0.98      0.98      0.98      1065
weighted avg       0.98      0.98      0.98      1065



In [29]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.9588014981273408
Confusion matrix
[[ 92   4]
 [  7 164]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.93      0.96      0.94        96
           1       0.98      0.96      0.97       171

    accuracy                           0.96       267
   macro avg       0.95      0.96      0.96       267
weighted avg       0.96      0.96      0.96       267



In [30]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(92, 4, 6, 165)

In [31]:
model_class.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 19ms/step


array([[0.9998925]], dtype=float32)

In [32]:
model_class.predict(
    np.array([nlp('чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 17ms/step


array([[0.9989486]], dtype=float32)

In [33]:
model_class.predict(
    np.array([nlp('поддерживать внутреннюю структуру').vector]))


1/1 [==============================] - 0s 18ms/step


array([[0.9999097]], dtype=float32)

In [34]:
model_class.save('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')
